In [1]:
import os
import numpy as np
import h5py
import scipy.io
np.random.seed(7) # for reproducibility

import keras.backend as K
from keras.models import Model, load_model

import matplotlib.pyplot as plt
import matplotlib
#from concise.utils.plot import seqlogo, seqlogo_fig

import sys
sys.path.append("../Scripts/")
from IntegratedGradients import *
from util_funcs import *
from plotseqlogo import seqlogo, seqlogo_fig

Using TensorFlow backend.
../Scripts/plotseqlogo.py:41: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  min_coords = np.vstack(data.min(0) for data in polygons_data).min(0)
../Scripts/plotseqlogo.py:42: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  max_coords = np.vstack(data.max(0) for data in polygons_data).max(0)


In [2]:
### library to use R
%load_ext rpy2.ipython
%load_ext rmagic

/anaconda3/lib/python3.7/site-packages/IPython/extensions/rmagic.py:11: UserWarning: The rmagic extension in IPython has moved to `rpy2.ipython`, please see `rpy2` documentation.
  warnings.warn("The rmagic extension in IPython has moved to "


In [3]:
def load_data_SHBIME(path_to_data):

    data = h5py.File(path_to_data,'r')
    X_test_seq = np.transpose(np.array(data['test_in_seq']),axes=(0,2,1))
    X_test_annotation = np.transpose(np.array(data['test_in_annotation']),axes=(0,2,1))
    X_test_region = np.transpose(np.array(data['test_in_region']),axes=(0, 2, 1))
    y_test_RBP = np.array(data['test_out'])
    y_test_name= np.array(data['test_name'])
    y_train= np.array(data['train_out'])
    data.close()
    return X_test_seq, X_test_annotation, X_test_region, y_test_RBP, y_test_name, y_train

In [4]:
path_to_data = "/Users/storm/git/Multi-resBind/Data/"
path_to_datalow = path_to_data + "data_RBPslow_CapR.h5"
X_test_seq_low,X_test_annotation_low, X_test_region_low, y_test_RBP_low, y_test_name_low, y_train_low = load_data_SHBIME(path_to_datalow)
print (X_test_seq_low.shape)
print (X_test_annotation_low.shape)
print (X_test_region_low.shape)
print (y_test_RBP_low.shape)
print (y_test_name_low.shape)
print (y_train_low.shape)

(7969, 150, 4)
(7969, 150, 6)
(7969, 250, 4)
(7969, 27)
(7969,)
(56547, 27)


In [5]:
d = np.concatenate((X_test_seq_low, X_test_region_low[:, 50:200, :], X_test_annotation_low), axis=2)
print (d.shape)

(7969, 150, 14)


In [6]:
#load models and obtain prediction and integrated_gradients
path_to_model = "/Users/storm/git/Multi-resBind/results/loss_functions/"
path_to_model_loss_0= path_to_model + "m_resbind_loss_0_model.h5"
path_to_model_loss_1= path_to_model + "m_resbind_loss_1_model.h5"
path_to_model_loss_2= path_to_model + "m_resbind_loss_2_model.h5"
path_to_model_loss_3= path_to_model + "m_resbind_loss_3_model.h5"
path_to_model_loss_4= path_to_model + "m_resbind_loss_4_model.h5"

In [7]:
model_loss_0=load_model(path_to_model_loss_0, custom_objects={'precision': precision,'recall': recall }) 
pred_loss_0=model_loss_0.predict(d)

Instructions for updating:
Colocations handled automatically by placer.


2021-05-11 11:20:16,524 [WARNING] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


2021-05-11 11:20:17,664 [WARNING] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


In [8]:
model_loss_1=load_model(path_to_model_loss_1, custom_objects={'precision': precision,'recall': recall }) 
pred_loss_1=model_loss_1.predict(d)

In [9]:
model_loss_2=load_model(path_to_model_loss_2, custom_objects={'precision': precision,'recall': recall }, compile=False) 
pred_loss_2=model_loss_2.predict(d)

In [10]:
model_loss_3=load_model(path_to_model_loss_3, custom_objects={'precision': precision,'recall': recall }, compile=False) 
pred_loss_3=model_loss_3.predict(d)

In [11]:
model_loss_4=load_model(path_to_model_loss_4, custom_objects={'precision': precision,'recall': recall }, compile=False) 
pred_loss_4=model_loss_4.predict(d)

In [12]:
#RBPnames for each model
RBPnames_low=np.array(['MBNL1', 'P53_NONO', 'PUM2', 'QKI', 'AGO3', 'FUS', 'TAF15', 'ZFP36', 'DICER1', 'EIF3A', 'EIF3D', 'EIF3G', 'SSB', 'PAPD5', 'CPSF4', 'CPSF3', 'RTCB', 'FXR1', 'NOP58', 'NOP56', 'FBL', 'LIN28A', 'LIN28B', 'UPF1', 'G35', 'G45', 'XPO5'])

In [13]:
#number of tasks for each model
num_task_low=len(RBPnames_low)

In [14]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import numpy as np
from matplotlib.font_manager import FontProperties
import matplotlib as mpl
import matplotlib

In [15]:
y_true=y_test_RBP_low
type(y_true) == np.ndarray
y_pred_loss_0=pred_loss_0
y_pred_loss_1=pred_loss_1
y_pred_loss_2=pred_loss_2
y_pred_loss_3=pred_loss_3
y_pred_loss_4=pred_loss_4

In [16]:
precision_low = l = [None] * len(RBPnames_low)
recall_low = [None] * len(RBPnames_low)
average_precision_low = [None] * len(RBPnames_low)

fpr_low = [None] * len(RBPnames_low)
tpr_low = [None] * len(RBPnames_low)
roc_auc_low = [None] * len(RBPnames_low)
pr_auc_low = [None] * len(RBPnames_low)

for i in range(num_task_low):
    precision_low[i], recall_low[i], _ = precision_recall_curve(y_true[:, i],y_pred_loss_0[:, i])
    average_precision_low[i] = average_precision_score(y_true[:, i], y_pred_loss_0[:, i])
    pr_auc_low [i]= auc(recall_low[i],precision_low[i])
    fpr_low[i], tpr_low[i], _ = roc_curve(y_true[:, i], y_pred_loss_0[:, i])
    roc_auc_low[i] = auc(fpr_low[i], tpr_low[i])
average_precision_loss_0 = average_precision_low
roc_auc_loss_0 = roc_auc_low

In [17]:
precision_low = l = [None] * len(RBPnames_low)
recall_low = [None] * len(RBPnames_low)
average_precision_low = [None] * len(RBPnames_low)

fpr_low = [None] * len(RBPnames_low)
tpr_low = [None] * len(RBPnames_low)
roc_auc_low = [None] * len(RBPnames_low)
pr_auc_low = [None] * len(RBPnames_low)

for i in range(num_task_low):
    precision_low[i], recall_low[i], _ = precision_recall_curve(y_true[:, i],y_pred_loss_1[:, i])
    average_precision_low[i] = average_precision_score(y_true[:, i], y_pred_loss_1[:, i])
    pr_auc_low [i]= auc(recall_low[i],precision_low[i])
    fpr_low[i], tpr_low[i], _ = roc_curve(y_true[:, i], y_pred_loss_1[:, i])
    roc_auc_low[i] = auc(fpr_low[i], tpr_low[i])
average_precision_loss_1 = average_precision_low
roc_auc_loss_1 = roc_auc_low

In [18]:
precision_low = l = [None] * len(RBPnames_low)
recall_low = [None] * len(RBPnames_low)
average_precision_low = [None] * len(RBPnames_low)

fpr_low = [None] * len(RBPnames_low)
tpr_low = [None] * len(RBPnames_low)
roc_auc_low = [None] * len(RBPnames_low)
pr_auc_low = [None] * len(RBPnames_low)

for i in range(num_task_low):
    precision_low[i], recall_low[i], _ = precision_recall_curve(y_true[:, i],y_pred_loss_2[:, i])
    average_precision_low[i] = average_precision_score(y_true[:, i], y_pred_loss_2[:, i])
    pr_auc_low [i]= auc(recall_low[i],precision_low[i])
    fpr_low[i], tpr_low[i], _ = roc_curve(y_true[:, i], y_pred_loss_2[:, i])
    roc_auc_low[i] = auc(fpr_low[i], tpr_low[i])
average_precision_loss_2 = average_precision_low
roc_auc_loss_2 = roc_auc_low

In [19]:
precision_low = l = [None] * len(RBPnames_low)
recall_low = [None] * len(RBPnames_low)
average_precision_low = [None] * len(RBPnames_low)

fpr_low = [None] * len(RBPnames_low)
tpr_low = [None] * len(RBPnames_low)
roc_auc_low = [None] * len(RBPnames_low)
pr_auc_low = [None] * len(RBPnames_low)

for i in range(num_task_low):
    precision_low[i], recall_low[i], _ = precision_recall_curve(y_true[:, i],y_pred_loss_3[:, i])
    average_precision_low[i] = average_precision_score(y_true[:, i], y_pred_loss_3[:, i])
    pr_auc_low [i]= auc(recall_low[i],precision_low[i])
    fpr_low[i], tpr_low[i], _ = roc_curve(y_true[:, i], y_pred_loss_3[:, i])
    roc_auc_low[i] = auc(fpr_low[i], tpr_low[i])
average_precision_loss_3 = average_precision_low
roc_auc_loss_3 = roc_auc_low

In [20]:
precision_low = l = [None] * len(RBPnames_low)
recall_low = [None] * len(RBPnames_low)
average_precision_low = [None] * len(RBPnames_low)

fpr_low = [None] * len(RBPnames_low)
tpr_low = [None] * len(RBPnames_low)
roc_auc_low = [None] * len(RBPnames_low)
pr_auc_low = [None] * len(RBPnames_low)

for i in range(num_task_low):
    precision_low[i], recall_low[i], _ = precision_recall_curve(y_true[:, i],y_pred_loss_4[:, i])
    average_precision_low[i] = average_precision_score(y_true[:, i], y_pred_loss_4[:, i])
    pr_auc_low [i]= auc(recall_low[i],precision_low[i])
    fpr_low[i], tpr_low[i], _ = roc_curve(y_true[:, i], y_pred_loss_4[:, i])
    roc_auc_low[i] = auc(fpr_low[i], tpr_low[i])
average_precision_loss_4 = average_precision_low
roc_auc_loss_4 = roc_auc_low

In [21]:
import numpy as np
print("roc_auc_class_aware:{:.4f}".format (np.mean(roc_auc_loss_0)))
print("roc_auc_BCE:{:.4f}".format (np.mean(roc_auc_loss_1)))
print("roc_auc_WBCE:{:.4f}".format (np.mean(roc_auc_loss_2)))
print("roc_auc_focal_loss:{:.4f}".format (np.mean(roc_auc_loss_3)))
print("roc_auc_ASL:{:.4f}".format (np.mean(roc_auc_loss_4)))

roc_auc_class_aware:0.8966
roc_auc_BCE:0.8981
roc_auc_WBCE:0.8934
roc_auc_focal_loss:0.8993
roc_auc_ASL:0.8975


In [22]:
print("average_precision_class_aware: {:.4f}".format (np.mean(average_precision_loss_0)))
print("average_precision_BCE: {:.4f}".format (np.mean(average_precision_loss_1)))
print("average_precision_WBCE: {:.4f}".format (np.mean(average_precision_loss_2)))
print("average_precision_focal_loss: {:.4f}".format (np.mean(average_precision_loss_3)))
print("average_precision_ASL: {:.4f}".format (np.mean(average_precision_loss_4)))

average_precision_class_aware: 0.3720
average_precision_BCE: 0.3763
average_precision_WBCE: 0.3523
average_precision_focal_loss: 0.3745
average_precision_ASL: 0.3765


In [23]:
import pandas as pd
df_AP = pd.DataFrame(np.zeros(num_task_low*6).reshape(num_task_low,6),
                  columns=['RBPnames','class_aware(AP)','BCE(AP)', 'WBCE(AP)', 'focal_loss(AP)','ASL(AP)'])
df_AP.loc[:, 'RBPnames'] = RBPnames_low
df_AP.loc[:, 'class_aware(AP)'] = average_precision_loss_0
df_AP.loc[:, 'BCE(AP)'] = average_precision_loss_1
df_AP.loc[:, 'WBCE(AP)'] = average_precision_loss_2
df_AP.loc[:, 'focal_loss(AP)'] = average_precision_loss_3
df_AP.loc[:, 'ASL(AP)'] = average_precision_loss_4
#df_AP.to_csv('supplemental_file_7.csv', index = False)
df_AP.head ()

,RBPnames,class_aware(AP),BCE(AP),WBCE(AP),focal_loss(AP),ASL(AP)
0,MBNL1,0.943411,0.948576,0.944890,0.948926,0.950086
1,P53_NONO,0.467233,0.435732,0.450310,0.443206,0.434452
2,PUM2,0.749511,0.760336,0.710780,0.740404,0.729038
3,QKI,0.639772,0.615919,0.455915,0.577508,0.605029
4,AGO3,0.546083,0.573884,0.509247,0.562891,0.561148


In [24]:
df_AUROC = pd.DataFrame(np.zeros(num_task_low*6).reshape(num_task_low,6),
                  columns=['RBPnames','class_aware(AUROC)', 'BCE(AUROC)', 'WBCE(AUROC)','focal_loss(AUROC)','ASL(AUROC)'])
df_AUROC.loc[:, 'RBPnames'] = RBPnames_low
df_AUROC.loc[:, 'class_aware(AUROC)'] = roc_auc_loss_0
df_AUROC.loc[:, 'BCE(AUROC)'] = roc_auc_loss_1
df_AUROC.loc[:, 'WBCE(AUROC)'] = roc_auc_loss_2
df_AUROC.loc[:, 'focal_loss(AUROC)'] = roc_auc_loss_3
df_AUROC.loc[:, 'ASL(AUROC)'] = roc_auc_loss_4
#df_AUROC.to_csv('supplemental_file_6.csv', index = False)
df_AUROC.head ()

,RBPnames,class_aware(AUROC),BCE(AUROC),WBCE(AUROC),focal_loss(AUROC),ASL(AUROC)
0,MBNL1,0.984343,0.984776,0.983855,0.985634,0.986008
1,P53_NONO,0.939246,0.935732,0.943078,0.940454,0.936160
2,PUM2,0.964620,0.966910,0.956724,0.963666,0.963963
3,QKI,0.980422,0.985567,0.969175,0.983773,0.982006
4,AGO3,0.892207,0.898567,0.884084,0.897848,0.896870
